# Import dependencies

In [54]:
# !pip install transformers[sentencepiece]
# !pip install datasets
# !pip install fugashi ipadic
# !pip install unidic_lite
# !apt-get install git-lfs

# Data Preprocess

In [56]:
import json
import glob
def read_json_file(file_path):
    with open(file_path) as f:
        data = json.load(f)
    return data

In [57]:
bn_train_data_path = "/media/user/hdd/NLP/QA/JaQuAD-main/bn_dataset/bn/train"
bn_val_data_path = "/media/user/hdd/NLP/QA/JaQuAD-main/bn_dataset/bn/val"
bn_train_files = glob.glob(bn_train_data_path+"/*.json")
bn_val_files = glob.glob(bn_val_data_path+"/*.json")

In [58]:
bn_val_files

['/media/user/hdd/NLP/QA/JaQuAD-main/bn_dataset/bn/val/validation.json']

In [59]:
from itertools import chain

In [60]:
bn_train_files

['/media/user/hdd/NLP/QA/JaQuAD-main/bn_dataset/bn/train/train.json']

In [61]:
bn_val_files

['/media/user/hdd/NLP/QA/JaQuAD-main/bn_dataset/bn/val/validation.json']

In [62]:
bn_train_data_list= list(map(read_json_file, bn_train_files))
bn_val_data_list= list(map(read_json_file, bn_val_files))

In [63]:
bn_train_list = [i["data"] for i in bn_train_data_list]
bn_val_list = [i["data"] for i in bn_val_data_list]


In [64]:
bn_train_data = list(chain(* bn_train_list))

In [65]:
len(bn_train_data)

1252

In [66]:
len(bn_train_data)

1252

In [67]:
bn_val_data = list(chain(* bn_val_list))

In [68]:
len(bn_val_data)

1252

In [69]:
import random
all_data = bn_train_data + bn_val_data

In [70]:
random.shuffle(all_data)

In [71]:
len(all_data)

2504

In [72]:
percentage = int((len(all_data)*90)/100)
print(percentage)

2253


In [73]:
bn_train_data = all_data[:percentage]
bn_val_data = all_data[percentage:]

In [74]:
from pprint import pprint
for i in bn_train_data[:10]:
    pprint(i)

{'paragraphs': [{'context': 'প্রাচীন মিশর উত্তর আফ্রিকার পূর্বাঞ্চলের একটি '
                            'প্রাচীন সভ্যতা। নীল নদের নিম্নভূমি অঞ্চলে এই '
                            'সভ্যতা গড়ে ওঠে। এই অঞ্চলটি বর্তমানে মিশর '
                            'রাষ্ট্রের অধিগত। খ্রিষ্টপূর্ব ৩১৫০ অব্দ নাগাদ[1] '
                            'প্রথম ফারাওয়ের অধীনে উচ্চ ও নিম্ন মিশরের '
                            'রাজনৈতিক একীকরণের মাধ্যমে এই সভ্যতা এক সুসংহত রূপ '
                            'লাভ করে। এরপর তিন সহস্রাব্দ কাল ধরে চলে প্রাচীন '
                            'মিশরীয় সভ্যতার বিকাশপর্ব।[2] প্রাচীন মিশরের '
                            'ইতিহাস একাধিক স্থায়ী রাজ্য-এর ইতিহাসের একটি '
                            'সুশৃঙ্খলিত ধারা। মধ্যে মধ্যে বিভিন্ন সময়ে '
                            'রাজনৈতিক অস্থিরতার পর্ব দেখা দিয়েছিল। এই '
                            'পর্বগুলি অন্তর্বর্তী পর্ব নামে পরিচিত। নতুন '
                            'রাজ্যের সময়কাল এই সভ্যতার চূড়ান্ত বিকাশপর্ব। এর '
      

# Model And Tokenizer Initialization

In [79]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

model = AutoModelForQuestionAnswering.from_pretrained("/media/user/hdd/NLP/QA/JaQuAD-main/results/checkpoint-26500")
tokenizer = AutoTokenizer.from_pretrained("/media/user/hdd/NLP/QA/JaQuAD-main/results/checkpoint-26500/", use_fast=True)
# tokenizer.do_lower_case = True 

In [85]:
text = 'রাগবি লিগটি বেশিরভাগ নিউজিল্যান্ড এবং অস্ট্রেলিয়ায় বসবাসকারী সামোয়ানরা খেলে, প্রশংসাপত্র প্রয়োজন এনআরএল, সুপার লিগ এবং ঘরোয়া '

string_tokenized = tokenizer.encode(text)
print(string_tokenized)

[0, 266, 264, 1123, 268, 339, 268, 1612, 268, 279, 265, 292, 268, 666, 264, 298, 297, 268, 897, 268, 272, 267, 277, 264, 269, 267, 323, 402, 312, 514, 267, 286, 267, 266, 265, 272, 268, 277, 367, 277, 309, 1431, 264, 535, 264, 266, 289, 281, 264, 274, 288, 277, 367, 798, 264, 345, 265, 272, 379, 285, 267, 453, 312, 280, 264, 390, 267, 266, 285, 267, 674, 832, 358, 735, 8156, 16, 281, 276, 287, 264, 266, 339, 268, 298, 402, 312, 793, 288, 277, 367, 225, 2]


In [87]:
def data_preprocessing(dataset):
    contexts, questions, answers = [], [], []
    for group in dataset:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return contexts, questions, answers

In [88]:
train_contexts, train_questions, train_answers = data_preprocessing(bn_train_data)
test_contexts, test_questions, test_answers = data_preprocessing(bn_val_data)

print(len(train_answers))
squad_train = {'answers': train_answers,'context': train_contexts, 'question': train_questions}
squad_test = {'answers': test_answers,'context': test_contexts, 'question': test_questions}

2402


In [89]:
squad_train["answers"]

[{'answer_start': 62, 'text': 'নীল নদের'},
 {'answer_start': 17, 'text': '২৯ আগস্ট ৫৭০'},
 {'answer_start': 81, 'text': 'সিলেট জেলার জকিগঞ্জ উপজেলার ফুলতলী'},
 {'answer_start': 52, 'text': 'পালি'},
 {'text': 'শশাঙ্ক', 'answer_start': 33},
 {'text': 'শশাঙ্ক', 'answer_start': 33},
 {'answer_start': 156, 'text': 'বিপ্লব কুমার দেব'},
 {'answer_start': 31, 'text': 'রক্তরস'},
 {'answer_start': 1234, 'text': 'ইডেন কলেজে'},
 {'answer_start': 13, 'text': 'গৌতম বুদ্ধ'},
 {'answer_start': 15, 'text': 'পশ্চিমবঙ্গ'},
 {'answer_start': 151, 'text': 'শচীন বানশাল এবং বিন্নী বানশাল'},
 {'answer_start': 0, 'text': 'এশিয়া'},
 {'answer_start': 202,
  'text': 'যুক্তরাষ্ট্রের ওয়াশিংটন অঙ্গরাজ্যের রেডমন্ড শহরে'},
 {'answer_start': 98, 'text': '১৮৭৯'},
 {'answer_start': 0, 'text': '১৯৯২'},
 {'answer_start': 0, 'text': 'জোকো "জোকোউয়ি" উইদোদো'},
 {'answer_start': 47, 'text': '১৬ অগস্ট, ১৮৮৬'},
 {'answer_start': 1390, 'text': 'বেলগ্রেড'},
 {'answer_start': 214, 'text': 'তিন'},
 {'answer_start': 540, 'text': '

In [90]:
squad_train["question"]

['মিশরীয় সভ্যতা কোন নদীর তীরে গড়ে উঠেছিল ?',
 'নববী মুহাম্মদ কবে জন্মগ্রহণ করেন ?',
 'আব্দুল লতিফ বাংলাদেশের কোথায় জন্মগ্রহণ করেন ?',
 'ত্রিপিটক কোন ভাষায় রচিত ?',
 'বৃহৎ বঙ্গ বা বিশাল বাংলার প্রথম স্বাধীন রাজা কে ছিলেন ?',
 'বৃহৎ বঙ্গ বা বিশাল বাংলার প্রথম স্বাধীন রাজা কে ছিলেন ?',
 'ত্রিপুরার বর্তমান (২০১৯) মুখ্যমন্ত্রী কে ?',
 'রক্তের তরল হালকা হলুদাভ অংশকে কি বলা হয় ?',
 'মাস্টারদা সূর্য সেন কোন কলেজে পড়াশোনা করেন ?',
 'বৌদ্ধধর্মের প্রচারক কে ছিলেন ?',
 'নবদ্বীপ জায়গাটি কোন রাজ্যে অবস্থিত ?',
 'ভারতীয় ই-কমার্স প্রতিষ্ঠান ফ্লিপকার্টের মালিক কে ?',
 'কাতার কোন মহাদেশের অন্তর্গত ?',
 'মাইক্রোসফট কর্পোরেশনের সদর দপ্তর কোথায় ?',
 'আইনস্টাইন কোন সালে জন্মগ্রহন করেন ?',
 'কথাসাহিত্যিক হুমায়ুন আহমেদ কত সালে প্রথম জাতীয় চলচ্চিত্র পুরস্কার পেয়েছিলেন ?',
 'ইন্দোনেশিয়ার বর্তমান (২০১৯) রাষ্ট্রপতি কে ?',
 'রামকৃষ্ণ পরমহংসের তিরোধান দিবস কবে ?',
 'যুগোস্লাভিয়ার রাজধানী কোথায় ?',
 'আকারের উপর ভিত্তি করে প্লাস্টিক দূষণকে কয়টি শ্রেণীতে ভাগ করা যায় ?',
 'ইউয়ান রাজবংশের প্রতিষ্ঠাতা কে ?',
 'চীনে

In [91]:
squad_train["context"]

['প্রাচীন মিশর উত্তর আফ্রিকার পূর্বাঞ্চলের একটি প্রাচীন সভ্যতা। নীল নদের নিম্নভূমি অঞ্চলে এই সভ্যতা গড়ে ওঠে। এই অঞ্চলটি বর্তমানে মিশর রাষ্ট্রের অধিগত। খ্রিষ্টপূর্ব ৩১৫০ অব্দ নাগাদ[1] প্রথম ফারাওয়ের অধীনে উচ্চ ও নিম্ন মিশরের রাজনৈতিক একীকরণের মাধ্যমে এই সভ্যতা এক সুসংহত রূপ লাভ করে। এরপর তিন সহস্রাব্দ কাল ধরে চলে প্রাচীন মিশরীয় সভ্যতার বিকাশপর্ব।[2] প্রাচীন মিশরের ইতিহাস একাধিক স্থায়ী রাজ্য-এর ইতিহাসের একটি সুশৃঙ্খলিত ধারা। মধ্যে মধ্যে বিভিন্ন সময়ে রাজনৈতিক অস্থিরতার পর্ব দেখা দিয়েছিল। এই পর্বগুলি অন্তর্বর্তী পর্ব নামে পরিচিত। নতুন রাজ্যের সময়কাল এই সভ্যতার চূড়ান্ত বিকাশপর্ব। এর পরই ধীরে ধীরে মিশরীয় সভ্যতার পতন আরম্ভ হয়। প্রাচীন মিশরের ইতিহাসের শেষ পবে একাধিক বৈদেশিক শক্তি মিশর অধিকার করে নেয়। খ্রিষ্টপূর্ব ৩১ অব্দে আদি রোমান সাম্রাজ্য মিশর অধিকার করে এই দেশকে একটি রোমান প্রদেশে পরিণত করলে ফারাওদের শাসন আনুষ্ঠানিকভাবে সমাপ্ত হয়।[3]',
 '   মুহাম্মাদ[1] (২৯ আগস্ট ৫৭০[2] - ৮ জুন ৬৩২;[3] আরবি উচ্চারণ শুনতে ক্লিক করুন  মোহাম্মদ এবং মুহম্মদ নামেও পরিচিত; তুর্কি: মুহাম্মেদ), পূর্ণ ন

In [92]:
def preprocess_function(examples, tokenizer):
#     print(examples.keys())
#     exit()
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=512,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
#         print(answer)
        start_char = answer["answer_start"]
        end_char = answer["answer_start"] + len(answer["text"])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [94]:
tokenized_squad_train = preprocess_function(squad_train,tokenizer)
tokenized_squad_test = preprocess_function(squad_test,tokenizer)

In [96]:
from functools import partial
def generated_dict(tokenized_squad):
    tokenized_squad_data = list(map(lambda x: {
            "input_ids": x[0], 
            "attention_mask": x[1], 
            "start_positions": x[2], 
            "end_positions": x[3]
        },
        zip(
            tokenized_squad["input_ids"], 
            tokenized_squad["attention_mask"],
            tokenized_squad["start_positions"], 
            tokenized_squad["end_positions"]
        )
    ))
    return tokenized_squad_data


In [97]:
add_part = partial(generated_dict)
tokenized_squad_train_new = add_part(tokenized_squad_train)
tokenized_squad_val_new = add_part(tokenized_squad_test)

In [101]:
for i in tokenized_squad_train_new[:10]:
    print(i)

{'input_ids': [0, 274, 268, 1097, 289, 275, 480, 267, 714, 264, 278, 288, 269, 771, 289, 266, 303, 289, 266, 265, 820, 265, 643, 265, 296, 268, 272, 4685, 2, 2, 287, 267, 266, 264, 305, 289, 269, 293, 268, 1097, 582, 267, 382, 838, 267, 266, 268, 270, 264, 266, 285, 350, 266, 267, 273, 264, 370, 267, 581, 265, 266, 384, 268, 285, 267, 266, 264, 305, 289, 269, 480, 267, 714, 359, 297, 289, 272, 771, 265, 266, 297, 268, 274, 267, 1192, 350, 274, 268, 977, 267, 581, 265, 377, 480, 267, 714, 264, 1272, 356, 809, 315, 377, 977, 267, 6030, 268, 475, 267, 543, 264, 269, 265, 293, 268, 1097, 316, 264, 301, 267, 286, 267, 266, 265, 266, 497, 268, 482, 283, 345, 267, 266, 268, 301, 267, 1196, 350, 266, 267, 273, 471, 6788, 470, 267, 284, 297, 264, 298, 264, 284, 63, 21, 65, 285, 267, 432, 346, 264, 266, 264, 479, 356, 266, 497, 289, 269, 265, 623, 267, 305, 321, 297, 268, 274, 267, 269, 293, 268, 1097, 265, 266, 316, 264, 358, 392, 271, 268, 270, 335, 289, 1061, 265, 266, 293, 264, 306, 267, 503

In [102]:
len(tokenized_squad_train_new)

2402

In [104]:
for i in tokenized_squad_val_new:
    print(i)
    

{'input_ids': [0, 341, 268, 272, 268, 280, 267, 271, 268, 269, 268, 308, 264, 271, 289, 277, 309, 271, 264, 273, 264, 284, 268, 427, 346, 264, 271, 264, 575, 747, 265, 1444, 268, 271, 331, 4685, 2, 2, 287, 267, 277, 264, 272, 265, 280, 267, 286, 264, 412, 339, 268, 273, 264, 266, 265, 395, 490, 267, 298, 264, 269, 264, 954, 265, 395, 279, 264, 285, 268, 4799, 32, 19, 70, 34, 266, 322, 265, 277, 367, 393, 267, 277, 264, 269, 294, 268, 280, 264, 273, 265, 372, 264, 341, 264, 271, 1579, 264, 277, 356, 272, 289, 2943, 264, 266, 268, 266, 279, 268, 266, 276, 284, 267, 306, 265, 281, 264, 266, 264, 291, 289, 394, 281, 312, 298, 267, 266, 264, 274, 302, 265, 269, 283, 303, 268, 269, 268, 285, 267, 277, 264, 272, 265, 280, 267, 286, 268, 269, 268, 277, 367, 269, 5184, 268, 286, 268, 266, 285, 267, 266, 265, 280, 268, 323, 265, 269, 267, 286, 265, 266, 300, 264, 277, 536, 271, 267, 273, 285, 264, 418, 302, 265, 269, 283, 308, 289, 394, 265, 266, 384, 264, 300, 289, 266, 267, 423, 741, 309, 372,

In [106]:
from transformers import TrainingArguments, Trainer, DefaultDataCollator

data_collator = DefaultDataCollator()

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=44,
    per_device_eval_batch_size=16,
    num_train_epochs=300,
    weight_decay=0.01,
    eval_steps=1000,
    save_total_limit=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad_train_new,
    eval_dataset=tokenized_squad_val_new,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/home/user/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2402
  Num Epochs = 300
  Instantaneous batch size per device = 44
  Total train batch size (w. parallel, distributed & accumulation) = 44
  Gradient Accumulation steps = 1
  Total optimization steps = 16500


Step,Training Loss,Validation Loss
1000,0.020500,0.024596
2000,0.014800,0.009051
3000,0.013100,0.083566
4000,0.012000,0.088645
5000,0.011000,0.076335
6000,0.011100,0.106847
7000,0.010400,0.098923
8000,0.010600,0.095571
9000,0.009700,0.074631
10000,0.009700,0.068250


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 262
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_tokens_map.json
Deleting older checkpoint [results/checkpoint-500] due to args.save_total_limit
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.

Model weights saved in ./results/checkpoint-9500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-9500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-9500/special_tokens_map.json
Deleting older checkpoint [results/checkpoint-9000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 262
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-10000
Configuration saved in ./results/checkpoint-10000/config.json
Model weights saved in ./results/checkpoint-10000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-10000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-10000/special_tokens_map.json
Deleting older checkpoint [results/checkpoint-9500] due to args.save_total_limit
Saving model checkpoint to ./results/checkpoint-10500
Configuration saved in ./results/checkpoint-10500/config.json
Model weights saved in ./results/checkpoint-10500/pytorch_model.bin
tokenizer confi

TrainOutput(global_step=16500, training_loss=0.01164216694687352, metrics={'train_runtime': 5534.046, 'train_samples_per_second': 130.212, 'train_steps_per_second': 2.982, 'total_flos': 9.41486238830592e+16, 'train_loss': 0.01164216694687352, 'epoch': 300.0})

# Inference

In [25]:
from transformers import AutoModelForQuestionAnswering,AutoTokenizer,pipeline

model_path = "/media/user/hdd/NLP/QA/JaQuAD-main/results/checkpoint-16500"
model = AutoModelForQuestionAnswering.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

for answer, context, question in zip(squad_test["answers"], squad_test["context"], squad_test["question"]):
    QA = pipeline('question-answering', model=model, tokenizer=tokenizer)
    QA_input = {'question': question,'context':context}
    prediction = QA(QA_input)
    
    print(prediction)
    
    print("="*40)

NameError: name 'squad_test' is not defined

# Single Inference

In [4]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline



model = AutoModelForQuestionAnswering.from_pretrained("saiful9379/Bangla_Roberta_Question_and_Answer")
tokenizer = AutoTokenizer.from_pretrained("saiful9379/Bangla_Roberta_Question_and_Answer")

context = "বাংলাদেশ ও ভারতের অনেক বৃহৎ নদী পূর্ব থেকে পশ্চিমে প্রবাহিত হয়ে বঙ্গোপসাগরে পতিত হয়েছে।\
তন্মধ্যে উত্তরদিক থেকে গঙ্গা, মেঘনা এবং ব্রহ্মপুত্র; দক্ষিণদিক থেকে মহানদী, গোদাবরী, কৃষ্ণা, ইরাবতী এবং কাবেরী নদী উল্লেখযোগ্য।\
৬৪ কিলোমিটারব্যাপী (৪০ মাইল) কৌম নদী সবচেয়ে ছোট নদী হিসেবে সরু খাল দিয়ে এবং ২,৯৪৮ কিলোমিটারব্যাপী (১,৮৩২ মাইল)\
বিশ্বের ২৮তম দীর্ঘ নদী হিসেবে ব্রহ্মপুত্র নদ বাংলাদেশ, চীন, নেপাল ও ভারতের মধ্য দিয়ে প্রবাহিত হয়ে বঙ্গোপসাগরে মিলিত হয়েছে।\
সুন্দরবন ম্যানগ্রোভ বনাঞ্চল গঙ্গা, ব্রহ্মপুত্র ও মেঘনা নদীর ব-দ্বীপকে ঘিরে গঠিত হয়েছে। মায়ানমারের (সাবেক বার্মা) ইরাওয়াদি (সংস্কৃত ইরাবতী)\
নদীও এ উপসাগরে মিলিত হয়েছে এবং একসময় গভীর ও ঘন ম্যানগ্রোভ বনাঞ্চলের সৃষ্টি করেছিল।"
 
question = "ব্রহ্মপুত্র নদের মোট দৈর্ঘ্য কত ?"

QA = pipeline('question-answering', model=model, tokenizer=tokenizer)
QA_input = {'question': question,'context':context}

prediction = QA(QA_input)
print(prediction)

{'score': 1.0, 'start': 294, 'end': 310, 'answer': '২,৯৪৮ কিলোমিটারব'}


In [24]:
import torch
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



context = "বাংলাদেশ ও ভারতের অনেক বৃহৎ নদী পূর্ব থেকে পশ্চিমে প্রবাহিত হয়ে বঙ্গোপসাগরে পতিত হয়েছে।\
তন্মধ্যে উত্তরদিক থেকে গঙ্গা, মেঘনা এবং ব্রহ্মপুত্র; দক্ষিণদিক থেকে মহানদী, গোদাবরী, কৃষ্ণা, ইরাবতী এবং কাবেরী নদী উল্লেখযোগ্য।\
৬৪ কিলোমিটারব্যাপী (৪০ মাইল) কৌম নদী সবচেয়ে ছোট নদী হিসেবে সরু খাল দিয়ে এবং ২,৯৪৮ কিলোমিটারব্যাপী (১,৮৩২ মাইল)\
বিশ্বের ২৮তম দীর্ঘ নদী হিসেবে ব্রহ্মপুত্র নদ বাংলাদেশ, চীন, নেপাল ও ভারতের মধ্য দিয়ে প্রবাহিত হয়ে বঙ্গোপসাগরে মিলিত হয়েছে।\
সুন্দরবন ম্যানগ্রোভ বনাঞ্চল গঙ্গা, ব্রহ্মপুত্র ও মেঘনা নদীর ব-দ্বীপকে ঘিরে গঠিত হয়েছে। মায়ানমারের (সাবেক বার্মা) ইরাওয়াদি (সংস্কৃত ইরাবতী)\
নদীও এ উপসাগরে মিলিত হয়েছে এবং একসময় গভীর ও ঘন ম্যানগ্রোভ বনাঞ্চলের সৃষ্টি করেছিল।"
 
question = "ব্রহ্মপুত্র নদের মোট দৈর্ঘ্য কত ?"

model = AutoModelForQuestionAnswering.from_pretrained("saiful9379/Bangla_Roberta_Question_and_Answer").to(device)
tokenizer = AutoTokenizer.from_pretrained("saiful9379/Bangla_Roberta_Question_and_Answer")


input_ids = tokenizer.encode(question, context)
tokens = tokenizer.convert_ids_to_tokens(input_ids)

output = model(torch.tensor([input_ids]).to(device))

argmax_start = torch.argmax(output['start_logits']).to(device)
argmax_end = torch.argmax(output['end_logits']).to(device)
print("="*40)
print("Question: ", question)
print("Context:", context)
print("-"*40)
print("Predicted answer: ", "".join(tokens[argmax_start:argmax_end]))

predicted_string = "".join(tokens[argmax_start:argmax_end])

token_id = tokenizer.encode(predicted_string)
print(argmax_start, argmax_end)
answer_tokens = tokens[argmax_start: argmax_end+1]

answer = tokenizer.decode(tokenizer.convert_tokens_to_ids(answer_tokens))

print("answer : ", answer)

# random_data = jaquad_data["validation"][random.randint(0, len(jaquad_data["validation"]))]
# from_data_to_answer(model, tokenizer, random_data)

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[-16.3680, -17.3164, -17.7502, -18.2334, -17.5051, -17.6700, -18.8044,
         -17.0317, -17.5096, -19.1373, -18.0707, -18.0804, -18.6880, -17.8809,
         -18.5109, -18.2540, -18.3011, -17.8451, -18.6201, -17.4243, -19.2553,
         -17.7516, -18.9651, -18.4140, -18.7526, -18.6903, -18.5156, -16.8956,
         -17.9776, -15.9104, -17.5950, -18.4859, -18.6443, -19.7269, -18.9075,
         -18.8117, -17.3282, -18.9628, -18.9563, -19.4569, -15.8712, -18.5416,
         -18.6439, -14.6385, -17.8338, -16.2377, -17.7837, -18.2652, -17.1790,
         -18.0542, -17.8363, -16.9750, -17.9468, -17.1633, -18.5397, -17.7660,
         -18.7768, -17.0808, -17.3248, -16.9080, -17.6782, -17.6809, -19.8366,
         -18.1068, -17.5276, -16.2343, -17.6140, -15.3399, -17.5103, -16.3853,
         -15.9636, -16.8899, -17.3155, -17.4412, -16.4900, -18.3246, -17.2063,
         -17.6613, -18.0371, -19.1178, -17.3419, -18.0175, -15.7748, -16.0610,

# Evalution

In [14]:
import collections
from typing import Any, Dict, Iterator, List, Tuple, Union
def compute_f1_score(ground_truth_values: List[str],
                     prediction_values: List[str]) -> float:
    '''Compute f1 score comparing two list of values.'''
    common = (
        collections.Counter(prediction_values) &
        collections.Counter(ground_truth_values))
    num_same = sum(common.values())

    # No answer case.
    if not ground_truth_values or not prediction_values:
        return int(ground_truth_values == prediction_values)

    if num_same == 0:
        return 0.

    precision = 1.0 * num_same / len(prediction_values)
    recall = 1.0 * num_same / len(ground_truth_values)
    f1_score = (2 * precision * recall) / (precision + recall)
    return f1_score


def char_f1_score(prediction: str, ground_truth: str) -> float:
    '''Character F1 score.'''
    prediction_tokens = prediction.split()
    ground_truth_tokens = ground_truth.split()

    # F1 by character
    prediction_char = []
    for tok in prediction_tokens:
        prediction_char.extend(list(tok))

    ground_truth_char = []
    for tok in ground_truth_tokens:
        ground_truth_char.extend(list(tok))
    return compute_f1_score(ground_truth_char, prediction_char)

# Reference

In [13]:

# https://medium.com/analytics-vidhya/create-a-tokenizer-and-train-a-huggingface-roberta-model-from-scratch-f3ed1138180c